In [22]:
import getpass
import os
import sys
sys.path.append(os.path.abspath('../backend'))

from langchain_community.graphs import Neo4jGraph
from graphdatascience import GraphDataScience
import pandas as pd
import numpy as np
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from src.llm import get_llm

import os
import getpass
from neo4j import GraphDatabase, Result
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tiktoken
import numpy as np
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Neo4jVector
from langchain_community.graphs import Neo4jGraph
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
# from llama_index.core.schema import TextNode
# from llama_index.core.vector_stores.utils import node_to_metadata_dict
# from llama_index.vector_stores.neo4jvector import Neo4jVectorStore
# from llama_index.core import VectorStoreIndex
from tqdm import tqdm
from src.shared.common_fn import load_embedding_model

from langchain_openai import AzureChatOpenAI
from typing import Dict, Any
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
from src.llm import get_llm

In [34]:
os.environ["NEO4J_URI"] = ""
os.environ["NEO4J_USERNAME"] = ""
os.environ["NEO4J_PASSWORD"] = ""

url = os.environ["NEO4J_URI"]
username =  os.environ["NEO4J_USERNAME"]
password = os.environ["NEO4J_PASSWORD"] 

os.environ["AZURE_OPENAI_API_KEY"] = ""
os.environ["AZURE_OPENAI_ENDPOINT"] = ""

llm = AzureChatOpenAI(temperature=0,azure_deployment="gpt-35",
    api_version="2024-02-15-preview")

In [35]:
gds = GraphDataScience(
    os.environ["NEO4J_URI"],
    auth=(os.environ["NEO4J_USERNAME"], os.environ["NEO4J_PASSWORD"])
)

graph = Neo4jGraph() 

driver = GraphDatabase.driver(os.environ["NEO4J_URI"], auth=(os.environ["NEO4J_USERNAME"], os.environ["NEO4J_PASSWORD"]))

# Search Methods

In [5]:
## local search

In [36]:
embeddings, dimension = load_embedding_model("sentence_transformer")

In [37]:
lc_retrieval_query = """
WITH collect(node) AS nodes
UNWIND nodes AS n
MATCH (n)<-[:HAS_ENTITY]-(c:Chunk)
WITH c, count(distinct n) AS freq, nodes
ORDER BY freq DESC
WITH collect({chunkText: c.text}) AS text_ref, nodes
UNWIND nodes AS n
MATCH (n)-[:IN_COMMUNITY]->(c:`__Community__`)
WITH text_ref,collect({community:c.summary}) as community,nodes
UNWIND nodes AS n
MATCH (n)-[r]-(m)
WITH text_ref, community, collect([n.id, TYPE(r), m.id, CASE 
    WHEN (n)-[r]->(m) THEN "outgoing"
    WHEN (n)<-[r]-(m) THEN "incoming"
    ELSE "undirected"
END]) AS Triples
RETURN {Chunks: text_ref, Community: community } AS text, 1.0 AS score, {Triples: Triples} AS metadata
"""

In [38]:
existing_graph= Neo4jVector.from_existing_graph(
            embedding = embeddings,
            url=url,
            username=username,
            password=password,
            index_name="entity",
            node_label="__Entity__",
            text_node_properties=["id"],
            embedding_node_property="embedding",
            retrieval_query=lc_retrieval_query 
        )

In [39]:
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain

In [59]:
prompt_template = """
You are an assistant tasked with extracting answers from the retrieve from the local_retrieval. The answer contains triplates as well as infomation of the nodes in chunks_details and community_summary.
Triplates : {triples} from Triplates Understand the 'outgoing' are outgoing relations and 'incoming' are incoming relations
and refer this : {chunks} for further reference based on above information.
 provide a concise and accurate answer to the question:
"{question}"
Provide the correct information in detailed explaination. If the context does not contain sufficient information to answer the question, respond with "The answer is not in the context."
"""

prompt = PromptTemplate(input_variables=["triples", "chunks", "question"], template=prompt_template)


def create_contextual_extraction_chain():
    return LLMChain(llm=llm, prompt=prompt)


# Example usage

question = "Who killed Dumbledore?"
docs = existing_graph.similarity_search(question,k=1)
chain = create_contextual_extraction_chain()

for doc in docs:
    metadata = doc.metadata
    page_content = doc.page_content
    
answer = chain.run(triples=metadata,chunks=page_content, question=question)
print(answer)

Based on the given information, the answer to the question "Who killed Dumbledore?" is not in the context.


In [60]:
docs

[Document(metadata={'Triples': [['Dumbledore', 'HAS_ENTITY', '18c2734716cad5d4df0bc03052857103249f330c', 'incoming'], ['Dumbledore', 'HAS_ENTITY', 'eb31684a257572d0260ab0d5dd15c148e377e5dc', 'incoming'], ['Dumbledore', 'HAS_ENTITY', 'b5c5a7c16c8449ac670e6e581d1077d73403b0e5', 'incoming'], ['Dumbledore', 'HAS_ENTITY', '3a6ea1c351161c3dd308263eded1a60f9e8d15d3', 'incoming'], ['Dumbledore', 'HAS_ENTITY', '98fcf19af9f513628792c4d1888c1f1d85f9add8', 'incoming'], ['Dumbledore', 'HAS_SUPPORTER', 'Voldemort', 'incoming'], ['Dumbledore', 'HAS_SUPPORTER', 'Harry', 'incoming'], ['Dumbledore', 'IN_COMMUNITY', '0-24', 'outgoing']]}, page_content="Community:\n- {'community': 'Voldemort is supported by Nagini, Snape, and Dumbledore, forming a network of allies.'}\nChunks:\n- {'chunkText': ' was in her vault at  Gringotts bank. She is very concerned about anything else they might have taken.  Dobby, the Malfoys’ former house-elf, helps Harry and his friends to escape, along  with Ollivander the wand m

In [ ]:
## Global search 

In [43]:
REDUCE_SYSTEM_PROMPT = """
---Role---
You are a helpful assistant responding to questions about a dataset by synthesizing perspectives from multiple analysts.
---Goal---
Generate a response of the target length and format that responds to the user's question, summarize all the reports from multiple analysts who focused on different parts of the dataset.
If you don't know the answer or if the provided reports do not contain sufficient information to provide an answer, just say so. Do not make anything up.
The final response should remove all irrelevant information from the analysts' reports and merge the cleaned information into a comprehensive answer that provides explanations of all the key points and implications appropriate for the response length and format.
Add sections and commentary to the response as appropriate for the length and format. Style the response in markdown.
The response shall preserve the original meaning and use of modal verbs such as "shall", "may" or "will".

**Do not list more than 5 record ids in a single reference**. Instead, list the top 5 most relevant record ids and add "+more" to indicate that there are more.

Do not include information where the supporting evidence for it is not provided.

---Target response length and format---
{response_type}

---Analyst Reports---

{report_data}

Add sections and commentary to the response as appropriate for the length and format. Style the response in markdown.
"""

reduce_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            REDUCE_SYSTEM_PROMPT,
        ),
        (
            "human",
            "{question}",
        ),
    ]
)
reduce_chain = reduce_prompt | llm | StrOutputParser()


In [44]:
MAP_SYSTEM_PROMPT = """
---Role---

You are a helpful assistant responding to questions about data in the tables provided.


---Goal---

Generate a response consisting of a list of key points that responds to the user's question, summarizing all relevant information in the input data tables.

You should use the data provided in the data tables below as the primary context for generating the response.
If you don't know the answer or if the input data tables do not contain sufficient information to provide an answer, just say so. Do not make anything up.

Each key point in the response should have the following element:
- Description: A comprehensive description of the point.
- Importance Score: An integer score between 0-100 that indicates how important the point is in answering the user's question. An 'I don't know' type of response should have a score of 0.

The response should be JSON formatted as follows:
{{
    "points": [
        {{"description": "Description of point 1 [Data: Reports (report ids)]", "score": score_value}},
        {{"description": "Description of point 2 [Data: Reports (report ids)]", "score": score_value}}
    ]
}}

The response shall preserve the original meaning and use of modal verbs such as "shall", "may" or "will".

Points supported by data should list the relevant reports as references as follows:
"This is an example sentence supported by data references [Data: Reports (report ids)]"

**Do not list more than 5 record ids in a single reference**. Instead, list the top 5 most relevant record ids and add "+more" to indicate that there are more.

For example:
"Person X is the owner of Company Y and subject to many allegations of wrongdoing [Data: Reports (2, 7, 64, 46, 34, +more)]. He is also CEO of company X [Data: Reports (1, 3)]"

where 1, 2, 3, 7, 34, 46, and 64 represent the id (not the index) of the relevant data report in the provided tables.

Do not include information where the supporting evidence for it is not provided.


---Data tables---

{context_data}


"""

map_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            MAP_SYSTEM_PROMPT,
        ),
        (
            "human",
            "{question}",
        ),
    ]
)

map_chain = map_prompt | llm | StrOutputParser()

In [56]:
response_type: str = "multiple paragraphs"


def global_retriever(query: str,  response_type: str = response_type) -> str:
    community_data = graph.query(
        """
        CALL db.index.fulltext.queryNodes("community_keyword", $query) YIELD node, score RETURN node.summary as output LIMIT 10
    """,
        params={"query":query},
    )
    
    intermediate_results = []
    for community in range(len(community_data)):
        intermediate_response = map_chain.invoke(
            {"question": query, "context_data": community_data[community]["output"]}
        )
        intermediate_results.append(intermediate_response)
    final_response = reduce_chain.invoke(
        {
            "report_data": intermediate_results,
            "question": query,
            "response_type": response_type,
        }
    )
    return final_response

In [46]:
response_type: str = "multiple paragraphs"


def global_retriever(query: str, level: int, response_type: str = response_type) -> str:
    community_data = graph.query(
        """
    MATCH (c:__Community__)
    WHERE c.level = $level
    RETURN c.summary AS output LIMIT 25
    """,
        params={"level": level},
    )
    
    intermediate_results = []
    for community in range(len(community_data)):
        intermediate_response = map_chain.invoke(
            {"question": query, "context_data": community_data[community]["output"]}
        )
        intermediate_results.append(intermediate_response)
    final_response = reduce_chain.invoke(
        {
            "report_data": intermediate_results,
            "question": query,
            "response_type": response_type,
        }
    )
    return final_response

In [48]:
print(global_retriever("Who killed Dumbledore?",1))

I'm sorry, but I don't have any information about who killed Snape in the provided data tables. I have reviewed multiple analyst reports, and none of them contain any information regarding Snape's killer. It seems that the dataset does not include this specific information.

It is important to note that the absence of information about Snape's killer in the dataset does not necessarily mean that it is impossible to determine. It may be that the dataset does not include the necessary details or that the analysts did not focus on this particular aspect.

If you have any other questions or if there is any other information you would like me to look for in the dataset, please let me know and I will do my best to assist you.


In [ ]:
## Drift Search

In [64]:
from typing import Optional
from pydantic import BaseModel, Field


class Answer(BaseModel):
    """Information about answer"""
    answer: Optional[str] = Field(default=None, description="The answer returned from any search methods ")
    additional_info: Optional[str] = Field(
        default=None, description="Any imporant additional infomation retrieved during global search"
    )
    

In [66]:
from typing import Optional

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from pydantic import BaseModel, Field

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert in extracting answer. there are two answers passed to a {question} compare both and extract proper answer for the question , with detailed explanation  present in any of them "
            "Only extract relevant information from the text with some addition proof for the answer, answer should be details. specific  "
            "If you do not know the value of an attribute asked to extract, "
            "return null for the attribute's value.",
        ),
        ("human", "Global Search Answer : {global_answer}, Local Search Answer : {local_answer}")
    ]
)

In [67]:
structured_llm = llm.with_structured_output(schema=Answer)

In [55]:
question = "Who killed Dumbledore?"
global_answer = global_retriever(question,1)
docs = existing_graph.similarity_search(question,k=1)
chain = create_contextual_extraction_chain()

for doc in docs:
    metadata = doc.metadata
    page_content = doc.page_content
    
local_answer = chain.run(triples=metadata,chunks=page_content, question=question)


prompt = prompt_template.invoke({"global_answer": global_answer,"local_answer":local_answer,"question":question})
structured_llm.invoke(prompt)


ERROR:neo4j.io:Failed to read from defunct connection IPv4Address(('54.89.254.144', 7687)) (ResolvedIPv4Address(('54.89.254.144', 7687)))
ERROR:neo4j.io:Failed to read from defunct connection IPv4Address(('54.89.254.144', 7687)) (ResolvedIPv4Address(('54.89.254.144', 7687)))


Answer(answer="The available data does not provide any insights into the identity of Dumbledore's killer.", additional_info=None)

In [69]:
question = "who killed Sanpe?"
global_answer = global_retriever(question)
docs = existing_graph.similarity_search(question,k=1)

for doc in docs:
    metadata = doc.metadata
    page_content = doc.page_content
    
local_answer = chain.run(triples=metadata,chunks=page_content, question=question)


prompt = prompt_template.invoke({"global_answer": global_answer,"local_answer":local_answer,"question":question})
structured_llm.invoke(prompt)

Answer(answer="I'm sorry, but I don't have any information about who killed Snape in the provided data tables.", additional_info=None)